### TASK

Develop and deploy machine
learning models in any one of the following areas only and analyse the results.

##### Public Transport

Project questions could be: (this is a small, suggested, sample of questions, other questions may be more
appropriate to your project)

– How to measure similarity or dissimilarity between different clusters?
– Which clustering solution do you prefer, and why?
– How to analyse and investigate an inflation rate for a specific product?

You will present their findings and defend the results in the report (MS Doc). Your report should capture the
following aspects that are relevant to your project investigations.

i) A precise introduction, motivation, description of problem domain, project objectives and the
rationale for the chosen dataset in the above-mentioned areas.


ii) Which clustering algorithms would you consider for segmentation, and why? Explain the differences
between silhouette score and Davies-Bouldin index in the context of clustering. Compare the results
obtained from any two clustering algorithms from the chosen dataset.


iii) What insights can you derive from the initial exploration of the time series data based on the
provided topics? Describe any trends, seasonality, or anomalies observed. How did you determine
the appropriate parameters (p, d, q) for the ARIMA model. Evaluate the performance of the ARIMA
model in forecasting future values, highlighting any strengths and limitations based on your chosen
dataset.


iv) Interpret and justify the results based on the problem specification or project objectives by using
suitable visualizations. Comments and description of Python code, conclusions of the project should
be specified in the report as well as jupyter notebook. Citations and references should be in the
Harvard Style.


#### Note: You can choose two different datasets for task (ii) or task (iii) separately or one dataset for both tasks

### Part I: Data Loading and Cleaning

In [1]:
# Step 1: Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Suppressing potential future warnings for pandas operations
pd.options.mode.chained_assignment = None

# Step 2: Loading the Dataset
# Load the dataset
df = pd.read_csv('MetroPT3(AirCompressor).csv')

# Display the first few rows of the dataframe
df.head()

,Unnamed: 0,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,0,01/02/2020 00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1,0,1,1,0,1,1,1
1,10,01/02/2020 00:00,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1,0,1,1,0,1,1,1
2,20,01/02/2020 00:00,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1,0,1,1,0,1,1,1
3,30,01/02/2020 00:00,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1,0,1,1,0,1,1,1
4,40,01/02/2020 00:00,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1,0,1,1,0,1,1,1


In [2]:
# Checking the total number of rows in the dataset
total_rows = len(df)
print(f"Total number of rows in the dataset: {total_rows}")


Total number of rows in the dataset: 1048575


In [3]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [4]:
# Step 3: Data Cleaning
# Checking for missing values in the dataset
print(df.isnull().sum())


Unnamed: 0         0
timestamp          0
TP2                0
TP3                0
H1                 0
DV_pressure        0
Reservoirs         0
Oil_temperature    0
Motor_current      0
COMP               0
DV_eletric         0
Towers             0
MPG                0
LPS                0
Pressure_switch    0
Oil_level          0
Caudal_impulses    0
dtype: int64


In [5]:
# Fill missing values with the median:
df.fillna(df.median(numeric_only=True), inplace=True)

# Convert 'timestamp' column to datetime type (if not already)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(df.dtypes)  # To check the data types of each column

Unnamed: 0                  int64
timestamp          datetime64[ns]
TP2                       float64
TP3                       float64
H1                        float64
DV_pressure               float64
Reservoirs                float64
Oil_temperature           float64
Motor_current             float64
COMP                        int64
DV_eletric                  int64
Towers                      int64
MPG                         int64
LPS                         int64
Pressure_switch             int64
Oil_level                   int64
Caudal_impulses             int64
dtype: object


In [6]:
# Step 4: Basic Exploration
# Descriptive statistics for numeric columns
df.describe()

,Unnamed: 0,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,5.242870e+06,1.436824e+00,8.963460e+00,7.473877e+00,8.626027e-02,8.964050e+00,6.112691e+01,1.980451e+00,8.266648e-01,1.692192e-01,9.158286e-01,8.218859e-01,2.446177e-03,9.892397e-01,9.855566e-01,9.112863e-01
std,3.026977e+06,3.290119e+00,6.324664e-01,3.402957e+00,4.437627e-01,6.317086e-01,6.690040e+00,2.323514e+00,3.785370e-01,3.749456e-01,2.776448e-01,3.826090e-01,4.939833e-02,1.031724e-01,1.193097e-01,2.843302e-01
min,0.000000e+00,-3.200000e-02,7.300000e-01,-3.600000e-02,-3.200000e-02,7.120000e-01,1.540000e+01,2.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.621435e+06,-1.400000e-02,8.468000e+00,8.228000e+00,-2.400000e-02,8.470000e+00,5.620000e+01,4.000000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,5.242870e+06,-1.200000e-02,8.930000e+00,8.758000e+00,-2.200000e-02,8.930000e+00,6.030000e+01,4.250000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,7.864305e+06,-1.000000e-02,9.472000e+00,9.356000e+00,-1.800000e-02,9.472000e+00,6.560000e+01,3.825000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,1.048574e+07,1.067600e+01,1.030200e+01,1.028800e+01,9.844000e+00,1.030000e+01,8.312500e+01,9.295000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [7]:
# Unique values in 'COMP' column which indicates compressor status
print(df['COMP'].unique())

[1 0]


In [ ]:
# Plot the time series
plt.style.use('fivethirtyeight')
df.plot(subplots=True,
        layout=(6, 3),
        figsize=(22,22),
        fontsize=10, 
        linewidth=1,
        sharex=False,
        title='Visualization of the original Time Series')
plt.show()